# Trabalho de Visão Computacional

## Importação das Bibliotecas:

In [1]:
import torch
import numpy as np
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import warnings
import glob as gb
import os
from importlib import reload
import sys
from tqdm import tqdm
import pandas as pd
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from PIL import Image

from models.alexNet import MY_AlexNet
from models.vgg16 import MY_VGG16
from utils.utils import *
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score

warnings.simplefilter('ignore')
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


--------------------------------------

## Configuração dos Datasets:

* CIFAR-10:

In [2]:
training_data_C10 = datasets.CIFAR10(
    root="../data/datasets/",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data_C10 = datasets.CIFAR10(
    root="../data/datasets/",
    train=False,
    download=True,
    transform=ToTensor(),
)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
training_data_C10, test_data_C10

(Dataset CIFAR10
     Number of datapoints: 50000
     Root location: ../data/datasets/
     Split: Train
     StandardTransform
 Transform: ToTensor(),
 Dataset CIFAR10
     Number of datapoints: 10000
     Root location: ../data/datasets/
     Split: Test
     StandardTransform
 Transform: ToTensor())

* Building-vs-forests:

In [2]:
from zipfile import ZipFile

with ZipFile("../data/datasets/buildings-vs-forests/test_set.zip", 'r') as zip_ref:
    zip_ref.extractall("../data/datasets/buildings-vs-forests/")
    
with ZipFile("../data/datasets/buildings-vs-forests/traning_set.zip", 'r') as zip_ref:
    zip_ref.extractall("../data/datasets/buildings-vs-forests/")

In [3]:
tfm = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5],[0.5, 0.5, 0.5])   
])

In [4]:
trian_path = '../data/datasets/buildings-vs-forests/traning_set'
test_path = '../data/datasets/buildings-vs-forests/test_set'

training_data_BF = ImageFolder(trian_path, transform = tfm)
test_data_BF = ImageFolder(test_path, transform = tfm)

len_train = len(training_data_BF)
len_test = len(test_data_BF)

training_data_BF.class_to_idx
test_data_BF.class_to_idx

{'buildings': 0, 'forest': 1}

-----------------------

## Treinamento e Avaliação - CIFAR10:

### AlexNet:

* Variando Epócas:

In [3]:
epochs = [5, 10, 20, 50]
accuracys = []
errors = []

accuracy = 0
error = 999

for epoch in tqdm(epochs):
    batch_size = 64
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_AlexNet(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    for t in range(epoch):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test_for_class(test_dataloader, model, loss_fn, device)
    
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'5 Epochs': accuracys[0], '10 Epochs': accuracys[1], '20 Epochs': accuracys[2], '50 Epochs': accuracys[3]})
df.to_csv('../data/results/cifar10_alexNet_epochs.csv', index=False)
df

100%|██████████| 4/4 [22:02<00:00, 330.74s/it]


,5 Epochs,10 Epochs,20 Epochs,50 Epochs
0,1.0,0.0,0.132,0.140
1,0.0,1.0,0.000,0.304
2,0.0,0.0,0.000,0.169
3,0.0,0.0,0.000,0.000
4,0.0,0.0,0.293,0.002
5,0.0,0.0,0.104,0.001
6,0.0,0.0,0.000,0.000
7,0.0,0.0,0.000,0.000
8,0.0,0.0,0.000,0.220
9,0.0,0.0,0.934,0.783


* Variando Batch Size:

In [6]:
batch_sizes = [64, 128, 256]
accuracys = []
errors = []

accuracy = 0
error = 999

for batch_size in tqdm(batch_sizes):
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_AlexNet(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test_for_class(test_dataloader, model, loss_fn, device)
        
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Batch Size: 64': accuracys[0], 'Batch Size: 128': accuracys[1], 'Batch Size: 256': accuracys[2]})
df.to_csv('../data/results/cifar10_alexNet_batchSize.csv', index=False)
df

100%|██████████| 3/3 [31:12<00:00, 624.29s/it]


,Batch Size: 64,Batch Size: 128,Batch Size: 256
0,0.796,0.000,0.000
1,0.000,0.000,0.000
2,0.002,0.000,0.000
3,0.000,0.000,0.000
4,0.039,0.009,0.000
5,0.000,0.000,0.002
6,0.000,0.000,0.000
7,0.000,0.000,0.000
8,0.005,0.562,1.000
9,0.529,0.731,0.000


* Variando Taxa de Aprendizado:

In [8]:
learnings = [1e-2, 1e-3, 1e-4]
accuracys = []
errors = []

accuracy = 0
error = 999

for rate in tqdm(learnings):
    train_dataloader = DataLoader(training_data_C10, batch_size=64)
    test_dataloader = DataLoader(test_data_C10, batch_size=64)

    model = MY_AlexNet(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=rate)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test_for_class(test_dataloader, model, loss_fn, device)

    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Learning Rate: 1e-2': accuracys[0], 'Learning Rate: 1e-3': accuracys[1], 'Learning Rate: 1e-4': accuracys[2]})
df.to_csv('../data/results/cifar10_alexNet_learnRate.csv', index=False)
df

100%|██████████| 3/3 [38:12<00:00, 764.01s/it]


,Learning Rate: 1e-2,Learning Rate: 1e-3,Learning Rate: 1e-4
0,0.706,0.978,0.009
1,0.785,0.000,0.000
2,0.559,0.016,0.000
3,0.622,0.000,0.000
4,0.549,0.000,0.000
5,0.577,0.000,0.000
6,0.767,0.400,0.999
7,0.684,0.000,0.000
8,0.840,0.002,0.000
9,0.732,0.000,0.000


* Avaliando Melhor Configuração:

In [7]:
accuracys = []
errors = []

pred_list = []
label_list = []

train_loss = []
test_loss = []

accuracy = 0
error = 999

batch_size = 64
epochs = 50
learning = 1e-2

for i in tqdm(range(3)):
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_AlexNet(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning)

    epochs = 50
    for t in range(50):
        train_error = train(train_dataloader, model, loss_fn, optimizer, device)
        predicts, labels, accuracy, test_error = test_for_class(test_dataloader, model, loss_fn, device)

        train_loss.append(train_error)
        test_loss.append(test_error)

    predicts = np.concatenate(predicts).ravel().tolist()
    labels = np.concatenate(labels).ravel().tolist()

    pred_list.append(predicts)
    label_list.append(labels)

    accuracys.append(accuracy)
    errors.append(test_error)

aux_list = []
for i in train_loss:
    aux_list.append(i.item())

df_loss = pd.DataFrame({'Train_loss': aux_list, 'Test_loss': test_loss})
df_loss.to_csv('../data/results/cifar10_alexNet_loss.csv', index = False)

df_cm = pd.DataFrame({'Pred_1': pred_list[0], 'Label_1':label_list[0], 'Pred_2': pred_list[1], 'Label_2':label_list[1], 'Pred_3': pred_list[2], 'Label_3':label_list[2]})
df_cm.to_csv('../data/results/cifar10_alexNet_confusion_matrix.csv', index=False)

df = pd.DataFrame({'Iter 1': accuracys[0], 'Iter 2': accuracys[1], 'Iter 3': accuracys[2]})
df.to_csv('../data/results/cifar10_alexNet_bestConfig.csv', index=False)
df

100%|██████████| 3/3 [37:55<00:00, 758.51s/it]


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

### VGG16:

* Variando Épocas:

In [10]:
epochs = [5, 10, 20, 50]
accuracys = []
errors = []

accuracy = []
error = 999

for epoch in tqdm(epochs):
    batch_size = 64
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_VGG16(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    for t in range(epoch):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test_for_class(test_dataloader, model, loss_fn, device)
    
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'5 Epochs': accuracys[0], '10 Epochs': accuracys[1], '20 Epochs': accuracys[2], '50 Epochs': accuracys[3]})
df.to_csv('../data/results/cifar10_vgg16_epochs.csv', index=False)
df

100%|██████████| 4/4 [1:05:47<00:00, 986.94s/it] 


,5 Epochs,10 Epochs,20 Epochs,50 Epochs
0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.0,1.0
2,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0


* Variando Batch Size:

In [3]:
batch_sizes = [64, 128, 256]
accuracys = []
errors = []

accuracy = 0
error = 999

for batch_size in tqdm(batch_sizes):
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_VGG16(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test_for_class(test_dataloader, model, loss_fn, device)
        
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Batch Size: 64': accuracys[0], 'Batch Size: 128': accuracys[1], 'Batch Size: 256': accuracys[2]})
df.to_csv('../data/results/cifar10_vgg16_batchSize.csv', index=False)
df

100%|██████████| 3/3 [1:38:57<00:00, 1979.26s/it]


,Batch Size: 64,Batch Size: 128,Batch Size: 256
0,0.0,0.0,0.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0
3,0.0,0.0,0.0
4,0.0,1.0,0.0
5,0.0,0.0,0.0
6,0.0,0.0,0.0
7,0.0,0.0,0.0
8,0.0,0.0,0.0
9,0.0,0.0,0.0


* Variando Taxa de Aprendizado:

In [3]:
learnings = [1e-2, 1e-3, 1e-4]
accuracys = []
errors = []

accuracy = 0
error = 999

for rate in tqdm(learnings):
    train_dataloader = DataLoader(training_data_C10, batch_size=64)
    test_dataloader = DataLoader(test_data_C10, batch_size=64)

    model = MY_VGG16(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=rate)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        accuracy, error = test_for_class(test_dataloader, model, loss_fn, device)

    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Learning Rate: 1e-2': accuracys[0], 'Learning Rate: 1e-3': accuracys[1], 'Learning Rate: 1e-4': accuracys[2]})
df.to_csv('../data/results/cifar10_vgg16_learnRate.csv', index=False)
df

100%|██████████| 3/3 [1:57:06<00:00, 2342.05s/it]


,Learning Rate: 1e-2,Learning Rate: 1e-3,Learning Rate: 1e-4
0,0.0,0.0,0.0
1,0.0,1.0,0.0
2,0.0,0.0,0.0
3,0.0,0.0,0.0
4,0.0,0.0,0.0
5,1.0,0.0,0.0
6,0.0,0.0,1.0
7,0.0,0.0,0.0
8,0.0,0.0,0.0
9,0.0,0.0,0.0


* Avaliando Melhor Configuração:

In [ ]:
accuracys = []
errors = []

pred_list = []
label_list = []

train_loss = []
test_loss = []

accuracy = 0
error = 999

batch_size = 64
epochs = 50
learning = 1e-2

for i in tqdm(range(3)):
    train_dataloader = DataLoader(training_data_C10, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_C10, batch_size=batch_size)

    model = MY_VGG16(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning)

    epochs = 50
    for t in range(50):
        train_error = train(train_dataloader, model, loss_fn, optimizer, device)
        predicts, labels, accuracy, test_error = test_for_class(test_dataloader, model, loss_fn, device)

        train_loss.append(train_error)
        test_loss.append(test_error)

    predicts = np.concatenate(predicts).ravel().tolist()
    labels = np.concatenate(labels).ravel().tolist()

    pred_list.append(predicts)
    label_list.append(labels)

    accuracys.append(accuracy)
    errors.append(test_error)

aux_list = []
for i in train_loss:
    aux_list.append(i.item())

df_loss = pd.DataFrame({'Train_loss': aux_list, 'Test_loss': test_loss})
df_loss.to_csv('../data/results/cifar10_vgg16_loss.csv', index = False)

df_cm = pd.DataFrame({'Pred_1': pred_list[0], 'Label_1':label_list[0], 'Pred_2': pred_list[1], 'Label_2':label_list[1], 'Pred_3': pred_list[2], 'Label_3':label_list[2]})
df_cm.to_csv('../data/results/cifar10_vgg16_confusion_matrix.csv', index=False)

df = pd.DataFrame({'Iter 1': accuracys[0], 'Iter 2': accuracys[1], 'Iter 3': accuracys[2]})
df.to_csv('../data/results/cifar10_vgg16_bestConfig.csv', index=False)
df

------------------------

## Treinamento e Avaliação - Building-vs-Forest:

### AlexNet:

* Variando Épocas:

In [20]:
epochs = [5, 10, 20, 50]
accuracys = []
errors = []
f1s = []
recalls = []
precisions = []

accuracy = 0
error = 999

for epoch in tqdm(epochs):
    batch_size = 64
    train_dataloader = DataLoader(training_data_BF, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_BF, batch_size=batch_size)

    model = MY_AlexNet(n_classes = 2)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    for t in range(epoch):
        train(train_dataloader, model, loss_fn, optimizer, device)
        predicts, labels, accuracy, error = test_new_metrics(test_dataloader, model, loss_fn, device)
    
    predicts = np.concatenate(predicts).ravel().tolist()
    labels = np.concatenate(labels).ravel().tolist()
    
    
    f1s.append(f1_score(labels, predicts, average="micro"))
    recalls.append(recall_score(labels, predicts, average="micro"))
    precisions.append(precision_score(labels, predicts, average="micro"))
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Epochs': epochs, 'Accuracy': accuracys, 'Error': errors, 'F1 Score': f1s, 'Recall Score': recalls, 'Precision': precisions})
df.to_csv('../data/results/BF_alexNet_epochs.csv', index=False)
df

100%|██████████| 4/4 [06:30<00:00, 97.74s/it] 


,Epochs,Accuracy,Error,F1 Score,Recall Score,Precision
0,5,52.030735,0.692501,0.520307,0.520307,0.520307
1,10,52.030735,0.691036,0.520307,0.520307,0.520307
2,20,52.030735,0.691055,0.520307,0.520307,0.520307
3,50,52.469813,0.685558,0.524698,0.524698,0.524698


* Variando Batch Size:

In [22]:
batch_sizes = [64, 128, 256]
accuracys = []
errors = []
f1s = []
recalls = []
precisions = []

accuracy = 0
error = 999

for batch_size in tqdm(batch_sizes):
    train_dataloader = DataLoader(training_data_BF, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_BF, batch_size=batch_size)

    model = MY_AlexNet(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        predicts, labels, accuracy, error = test_new_metrics(test_dataloader, model, loss_fn, device)
    
    predicts = np.concatenate(predicts).ravel().tolist()
    labels = np.concatenate(labels).ravel().tolist()

    f1s.append(f1_score(labels, predicts, average="micro"))
    recalls.append(recall_score(labels, predicts, average="micro"))
    precisions.append(precision_score(labels, predicts, average="micro"))  
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Batch Size': batch_sizes, 'Accuracy': accuracys, 'Error': errors, 'F1 Score': f1s, 'Recall Score': recalls, 'Precision': precisions})
df.to_csv('../data/results/BF_alexNet_batchSize.csv', index=False)
df

100%|██████████| 3/3 [11:08<00:00, 222.75s/it]


,Batch Size,Accuracy,Error,F1 Score,Recall Score,Precision
0,64,52.030735,1.152476,0.520307,0.520307,0.520307
1,128,52.030735,0.883087,0.520307,0.520307,0.520307
2,256,52.030735,0.775018,0.520307,0.520307,0.520307


* Variando Taxa de Aprendizado:

In [23]:
learnings = [1e-2, 1e-3, 1e-4]
accuracys = []
errors = []
f1s = []
recalls = []
precisions = []

accuracy = 0
error = 999

for rate in tqdm(learnings):
    train_dataloader = DataLoader(training_data_BF, batch_size=64)
    test_dataloader = DataLoader(test_data_BF, batch_size=64)

    model = MY_AlexNet(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=rate)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        predicts, labels, accuracy, error = test_new_metrics(test_dataloader, model, loss_fn, device)
    
    predicts = np.concatenate(predicts).ravel().tolist()
    labels = np.concatenate(labels).ravel().tolist()

    f1s.append(f1_score(labels, predicts, average="micro"))
    recalls.append(recall_score(labels, predicts, average="micro"))
    precisions.append(precision_score(labels, predicts, average="micro"))
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Learning Rate': learnings, 'Accuracy': accuracys, 'Error': errors, 'F1 Score': f1s, 'Recall Score': recalls, 'Precision': precisions})
df.to_csv('../data/results/BF_alexNet_learnRate.csv', index=False)
df

100%|██████████| 3/3 [14:21<00:00, 287.29s/it]


,Learning Rate,Accuracy,Error,F1 Score,Recall Score,Precision
0,0.0100,52.030735,1.422985,0.520307,0.520307,0.520307
1,0.0010,52.030735,1.188879,0.520307,0.520307,0.520307
2,0.0001,52.030735,2.072739,0.520307,0.520307,0.520307


* Avaliando Melhor Configuração:

In [ ]:
accuracys = []
errors = []
f1s = []
recalls = []
precisions = []

pred_list = []
label_list = []

train_loss = []
test_loss = []

accuracy = 0
error = 999

batch_size = 64
epochs = 50
learning = 1e-2

for i in tqdm(range(3)):
    train_dataloader = DataLoader(training_data_BF, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_BF, batch_size=batch_size)

    model = MY_AlexNet(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning)

    epochs = 50
    for t in range(50):
        train_error = train(train_dataloader, model, loss_fn, optimizer, device)
        predicts, labels, accuracy, test_error = test_new_metrics(test_dataloader, model, loss_fn, device)

        train_loss.append(train_error)
        test_loss.append(test_error)
    
    predicts = np.concatenate(predicts).ravel().tolist()
    labels = np.concatenate(labels).ravel().tolist()

    pred_list.append(predicts)
    label_list.append(labels)

    f1s.append(f1_score(labels, predicts, average="micro"))
    recalls.append(recall_score(labels, predicts, average="micro"))
    precisions.append(precision_score(labels, predicts, average="micro"))
    accuracys.append(accuracy)
    errors.append(test_error)

aux_list = []
for i in train_loss:
    aux_list.append(i.item())

df_loss = pd.DataFrame({'Train_loss': aux_list, 'Test_loss': test_loss})
df_loss.to_csv('../data/results/bf_alexNet_loss.csv', index = False)

df_cm = pd.DataFrame({'Pred_1': pred_list[0], 'Label_1':label_list[0], 'Pred_2': pred_list[1], 'Label_2':label_list[1], 'Pred_3': pred_list[2], 'Label_3':label_list[2]})
df_cm.to_csv('../data/results/bf_alexNet_confusion_matrix.csv', index=False)

df = pd.DataFrame({'Accuracy': accuracys, 'Error': errors, 'F1 Score': f1s, 'Recall Score': recalls, 'Precision': precisions})
df.to_csv('../data/results/BF_alexNet_bestConfig.csv', index=False)
df

### VGG16:

* Variando Épocas:

In [4]:
epochs = [5, 10, 20, 50]
accuracys = []
errors = []
f1s = []
recalls = []
precisions = []

accuracy = 0
error = 999

for epoch in tqdm(epochs):
    batch_size = 64
    train_dataloader = DataLoader(training_data_BF, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_BF, batch_size=batch_size)

    model = MY_VGG16(n_classes = 2)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    for t in range(epoch):
        train(train_dataloader, model, loss_fn, optimizer, device)
        predicts, labels, accuracy, error = test_new_metrics(test_dataloader, model, loss_fn, device)
    
    predicts = np.concatenate(predicts).ravel().tolist()
    labels = np.concatenate(labels).ravel().tolist()
    
    f1s.append(f1_score(labels, predicts, average="micro"))
    recalls.append(recall_score(labels, predicts, average="micro"))
    precisions.append(precision_score(labels, predicts, average="micro"))
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Epochs': epochs, 'Accuracy': accuracys, 'Error': errors, 'F1 Score': f1s, 'Recall Score': recalls, 'Precision': precisions})
df.to_csv('../data/results/BF_vgg16_epochs.csv', index=False)
df

100%|██████████| 4/4 [11:58<00:00, 179.75s/it]


,Epochs,Accuracy,Error,F1 Score,Recall Score,Precision
0,5,52.030735,0.692524,0.520307,0.520307,0.520307
1,10,52.030735,0.692262,0.520307,0.520307,0.520307
2,20,52.030735,0.691415,0.520307,0.520307,0.520307
3,50,52.030735,0.690752,0.520307,0.520307,0.520307


* Variando Batch Size:

In [5]:
batch_sizes = [64, 128, 256]
accuracys = []
errors = []
f1s = []
recalls = []
precisions = []

accuracy = 0
error = 999

for batch_size in tqdm(batch_sizes):
    train_dataloader = DataLoader(training_data_BF, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_BF, batch_size=batch_size)

    model = MY_VGG16(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        predicts, labels, accuracy, error = test_new_metrics(test_dataloader, model, loss_fn, device)
    
    predicts = np.concatenate(predicts).ravel().tolist()
    labels = np.concatenate(labels).ravel().tolist()
    
    f1s.append(f1_score(labels, predicts, average="micro"))
    recalls.append(recall_score(labels, predicts, average="micro"))
    precisions.append(precision_score(labels, predicts, average="micro"))
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Batch Size': batch_sizes, 'Accuracy': accuracys, 'Error': errors, 'F1 Score': f1s, 'Recall Score': recalls, 'Precision': precisions})
df.to_csv('../data/results/BF_vgg16_batchSize.csv', index=False)
df

100%|██████████| 3/3 [27:30<00:00, 550.08s/it]


,Batch Size,Accuracy,Error,F1 Score,Recall Score,Precision
0,64,52.030735,0.790139,0.520307,0.520307,0.520307
1,128,52.030735,1.074370,0.520307,0.520307,0.520307
2,256,52.030735,1.811420,0.520307,0.520307,0.520307


* Variando Taxa de Aprendizado:

In [6]:
learnings = [1e-2, 1e-3, 1e-4]
accuracys = []
errors = []
f1s = []
recalls = []
precisions = []

accuracy = 0
error = 999

for rate in tqdm(learnings):
    train_dataloader = DataLoader(training_data_BF, batch_size=64)
    test_dataloader = DataLoader(test_data_BF, batch_size=64)

    model = MY_VGG16(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=rate)

    epochs = 50
    for t in range(50):
        train(train_dataloader, model, loss_fn, optimizer, device)
        predicts, labels, accuracy, error = test_new_metrics(test_dataloader, model, loss_fn, device)
    
    predicts = np.concatenate(predicts).ravel().tolist()
    labels = np.concatenate(labels).ravel().tolist()

    f1s.append(f1_score(labels, predicts, average="micro"))
    recalls.append(recall_score(labels, predicts, average="micro"))
    precisions.append(precision_score(labels, predicts, average="micro"))
    accuracys.append(accuracy)
    errors.append(error)

df = pd.DataFrame({'Learning Rate': learnings, 'Accuracy': accuracys, 'Error': errors, 'F1 Score': f1s, 'Recall Score': recalls, 'Precision': precisions})
df.to_csv('../data/results/BF_vgg16_learnRate.csv', index=False)
df

100%|██████████| 3/3 [44:42<00:00, 894.02s/it]


,Learning Rate,Accuracy,Error,F1 Score,Recall Score,Precision
0,0.0100,52.030735,1.235880,0.520307,0.520307,0.520307
1,0.0010,52.030735,0.787854,0.520307,0.520307,0.520307
2,0.0001,52.030735,2.105333,0.520307,0.520307,0.520307


* Avaliando Melhor Configuração:

In [22]:
accuracys = []
errors = []
f1s = []
recalls = []
precisions = []

pred_list = []
label_list = []

train_loss = []
test_loss = []

accuracy = 0
error = 999

batch_size = 64
epochs = 50
learning = 1e-2

for i in tqdm(range(3)):
    train_dataloader = DataLoader(training_data_BF, batch_size=batch_size)
    test_dataloader = DataLoader(test_data_BF, batch_size=batch_size)

    model = MY_VGG16(n_classes = 10)
    if torch.cuda.is_available():
        model.cuda()

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=learning)

    epochs = 50
    for t in range(50):
        train_error = train(train_dataloader, model, loss_fn, optimizer, device)
        predicts, labels, accuracy, test_error = test_new_metrics(test_dataloader, model, loss_fn, device)

        train_loss.append(train_error)
        test_loss.append(test_error)
    
    predicts = np.concatenate(predicts).ravel().tolist()
    labels = np.concatenate(labels).ravel().tolist()

    pred_list.append(predicts)
    label_list.append(labels)

    f1s.append(f1_score(labels, predicts, average="micro"))
    recalls.append(recall_score(labels, predicts, average="micro"))
    precisions.append(precision_score(labels, predicts, average="micro"))
    accuracys.append(accuracy)
    errors.append(test_error)

aux_list = []
for i in train_loss:
    aux_list.append(i.item())

df_loss = pd.DataFrame({'Train_loss': aux_list, 'Test_loss': test_loss})
df_loss.to_csv('../data/results/bf_vgg16_loss.csv', index = False)

df_cm = pd.DataFrame({'Pred_1': pred_list[0], 'Label_1':label_list[0], 'Pred_2': pred_list[1], 'Label_2':label_list[1], 'Pred_3': pred_list[2], 'Label_3':label_list[2]})
df_cm.to_csv('../data/results/bf_vgg16_confusion_matrix.csv', index=False)

df = pd.DataFrame({'Accuracy': accuracys, 'Error': errors, 'F1 Score': f1s, 'Recall Score': recalls, 'Precision': precisions})
df.to_csv('../data/results/BF_vgg16_bestConfig.csv', index=False)
df

100%|██████████| 3/3 [41:55<00:00, 838.65s/it]


,Accuracy,Error,F1 Score,Recall Score,Precision
0,52.030735,1.243675,0.520307,0.520307,0.520307
1,52.030735,1.253513,0.520307,0.520307,0.520307
2,52.030735,1.232855,0.520307,0.520307,0.520307


-------------------------------